In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### %cd /kaggle/working
%rm -rm LLaMA-Factory
!git clone https://github.com/hiyouga/LLaMA-Factory.git
%cd /kaggle/working/LLaMA-Factory
!pip install -e .[torch,bitsandbytes]

In [2]:
!pip install -r requirements.txt

In [4]:
import json

NAME = "llama-3"
AUTHOR = 'llama-factory'

with open('/kaggle/working/LLaMA-Factory/data/identity.json', 'r', encoding = 'utf-8') as f:
    dataset = json.load(f)


for sample in dataset:
    sample['output'] = sample['output'].replace("{{"+ "name" +"}}", NAME).replace("{{"+ "author" +"}}", AUTHOR)


with open('/kaggle/working/LLaMA-Factory/data/identiry.json', 'w', encoding = 'utf-8') as f:
    json.dump(dataset, f, indent = 2, ensure_ascii= False)

In [6]:
import json

arguments = dict(
    stage = 'sft',
    do_train = True,
    model_name_or_path = 'unsloth/llama-3-8b-Instruct-bnb-4bit',
    dataset = 'identity, alpaca_en_demo',
    template = 'llama3',
    finetuning_type = 'lora',
    lora_target = 'all',
    output_dir = 'llama3_lora',
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4,
    lr_scheduler_type = 'cosine',
    logging_steps = 10,
    warmup_ratio = 0.1,
    save_steps = 1000,
    learning_rate = 5e-5,
    num_train_epochs = 3.0,
    max_samples = 500,
    max_grad_norm = 1.0,
    quantization_bit = 4,
    loraplus_lr_ratio = 16.0,
    fp16 = True
)

json.dump(arguments, open('/kaggle/working/LLaMA-Factory/train_llama3.json', 'w', encoding = 'utf-8'), indent = 2)

!WANDB_API_KEY=0115cf8decaeb0bafd76e525335306858a1b377d llamafactory-cli train /kaggle/working/LLaMA-Factory/train_llama3.json

[INFO|2024-11-18 18:33:38] llamafactory.cli:157 >> Initializing distributed tasks at: 127.0.0.1:27054
W1118 18:33:40.149000 138027122063168 torch/distributed/run.py:779] 
W1118 18:33:40.149000 138027122063168 torch/distributed/run.py:779] *****************************************
W1118 18:33:40.149000 138027122063168 torch/distributed/run.py:779] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1118 18:33:40.149000 138027122063168 torch/distributed/run.py:779] *****************************************
[WARNING|2024-11-18 18:33:48] llamafactory.hparams.parser:162 >> We recommend enable `upcast_layernorm` in quantized training.
[WARNING|2024-11-18 18:33:48] llamafactory.hparams.parser:162 >> `ddp_find_unused_parameters` needs to be set as False for LoRA in DDP training.
[INFO|2024-11-18 18:33:48] llamafactory.hparams.parser:355 

In [7]:
!pip install git+https://github.com/hiyouga/LLaMA-Factory.git

  Cloning https://github.com/hiyouga/LLaMA-Factory.git to /tmp/pip-req-build-56aibdds
  Running command git clone --filter=blob:none --quiet https://github.com/hiyouga/LLaMA-Factory.git /tmp/pip-req-build-56aibdds
  Resolved https://github.com/hiyouga/LLaMA-Factory.git to commit 9c0f6556eeeef4f24b589c15aebfd68fd6f998c6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llamafactory: filename=llamafactory-0.9.1.dev0-py3-none-any.whl size=253345 sha256=a1faa35c608a291df5294579c4defb6f76d1725caed5e7087ec9cf4ed05938d7
  Stored in directory: /tmp/pip-ephem-wheel-cache-1gewaksw/wheels/6f/e2/1c/0951237e1114b7824788ea6e772820897ee0f6506afec796bb
Successfully built llamafactory
  Attempting uninstall: llamafactory
    Found existing installation: llamafactory 0.9.1.dev0
    Uninstalling llamafactory-0.9.1.dev0:
      Successfully uninstalled llamafactory-0.9.1.dev0


In [8]:
from llamafactory.chat import ChatModel
from llamafactory.extras.misc import torch_gc

%cd /kaggle/working/LLaMA-Factory

args = dict(
    model_name_or_path='unsloth/llama-3-8b-Instruct-bnb-4bit',
    adapter_name_or_path="llama3_lora",
    template='llama3',
    finetuning_type="lora",
    quantization_bit=4,
)

# هنگام بارگذاری مدل
chat_model = ChatModel(args)

message = []

print("Welcome to the CIL application .")
while True:
  query = input("\nUser:")
  if query.strip() == 'exit':
    break

  if query.strip() == 'clear':
    message = []
    torch_gc()
    print("History has been removed.")
    continue

  message.append({'role' : 'user', 'content' : query})
  print("Assistant : ", end = "", flush = True)

  response = ""
  for new_text in chat_model.stream_chat(message):
    print(new_text, end = "", flush = True)
    response += new_text

  print()
  message.append({'role' : "assistant", 'content' : response})


torch_gc()

[INFO|configuration_utils.py:672] 2024-11-18 19:06:45,501 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--unsloth--llama-3-8b-Instruct-bnb-4bit/snapshots/f296897830363557c84cc4a942c2cd1f91818ae4/config.json
[INFO|configuration_utils.py:739] 2024-11-18 19:06:45,502 >> Model config LlamaConfig {
  "_name_or_path": "unsloth/llama-3-8b-Instruct-bnb-4bit",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128009,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pad_token_id": 128255,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dt

/kaggle/working/LLaMA-Factory


[INFO|tokenization_utils_base.py:2214] 2024-11-18 19:06:45,574 >> loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--unsloth--llama-3-8b-Instruct-bnb-4bit/snapshots/f296897830363557c84cc4a942c2cd1f91818ae4/tokenizer.json
[INFO|tokenization_utils_base.py:2214] 2024-11-18 19:06:45,574 >> loading file tokenizer.model from cache at None
[INFO|tokenization_utils_base.py:2214] 2024-11-18 19:06:45,575 >> loading file added_tokens.json from cache at None
[INFO|tokenization_utils_base.py:2214] 2024-11-18 19:06:45,576 >> loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--unsloth--llama-3-8b-Instruct-bnb-4bit/snapshots/f296897830363557c84cc4a942c2cd1f91818ae4/special_tokens_map.json
[INFO|tokenization_utils_base.py:2214] 2024-11-18 19:06:45,576 >> loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--unsloth--llama-3-8b-Instruct-bnb-4bit/snapshots/f296897830363557c84cc4a942c2cd1f91818ae4/tokenizer_con

[INFO|2024-11-18 19:06:46] llamafactory.data.template:157 >> Replace eos token: <|eot_id|>


[INFO|configuration_utils.py:672] 2024-11-18 19:06:46,950 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--unsloth--llama-3-8b-Instruct-bnb-4bit/snapshots/f296897830363557c84cc4a942c2cd1f91818ae4/config.json
[INFO|configuration_utils.py:739] 2024-11-18 19:06:46,951 >> Model config LlamaConfig {
  "_name_or_path": "unsloth/llama-3-8b-Instruct-bnb-4bit",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128009,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pad_token_id": 128255,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dt

...


User: 你好，请问有什么可以帮助你


Assistant : 您好，我可以回答您的问题、提供语言翻译、生成文本、进行自然语言处理等。



User: Are you trained by OpenAI?


Assistant : No, I am {{name}}, an AI assistant developed by {{author}}.



User: exit
